In [ ]:
# specify the topic name here
topic_name = 'Political'

# entry folder for the data
# should be the form of 'data_{topic_name}/'
entry_folder = 'data_Political/'

In [ ]:
import os
import glob
import pandas as pd
from tqdm import tqdm

pd.options.mode.chained_assignment = None  # default='warn'

In [ ]:
def get_image_path(image_url, csv_path):
    image_name = image_url.split('/')[-1]
    image_path = './' + csv_path[:-4] + '/' + image_name
    return image_path

In [ ]:
df_csv = pd.DataFrame(columns=['hashtag', 'tweet_id', 'image_path', 'body_text'])
df_csv['tweet_id'] = df_csv['tweet_id'].astype('int64')

entries = os.listdir(entry_folder)
for hashtag in entries:
    if hashtag.startswith('.'): continue
    list_files = glob.glob(entry_folder + hashtag + '/*.csv')
    tweets_count = 0
    for csv_path in list_files:
        # print(csv_path)
        temp_df = pd.read_csv(csv_path)
        tweets_count += len(temp_df.index)
        temp_df.drop(columns=['tweet_url'], inplace=True)
        temp_df.rename({'text_data': 'body_text', 'media': 'image_path'}, axis='columns', inplace=True)
        temp_df.insert(loc=0, column='hashtag', value=hashtag)
        cols = temp_df.columns.tolist()
        # before: cols = ['hashtag', 'image_path', 'body_text', 'tweet_id']
        cols = cols[:1] + cols[-1:] + cols[1:-1]
        temp_df = temp_df[cols]
        temp_df['image_path'] = temp_df['image_path'].apply(get_image_path, csv_path=csv_path)
        temp_df['tweet_id'] = temp_df['tweet_id'].astype('int64')
        df_csv = df_csv.append(temp_df)
    print('{:>24s}: {:>5}'.format(hashtag, tweets_count))
print('Topic: {} - Total tweets: {:>5}'.format(topic_name, len(df_csv.index)))

In [ ]:
# remove tweets with invalid image path
df_csv['path_is_valid'] = df_csv['image_path'].apply(lambda x: 1 if os.path.isfile(x) else 0)
df_csv = df_csv[df_csv.path_is_valid.eq(1)]
df_csv.drop(columns=['path_is_valid'], inplace=True)

In [ ]:
df_csv.reset_index(drop=True, inplace=True)
df_csv

### Filter out images without text

In [ ]:
from opencv_text_detector import TextDetector

tqdm.pandas()
detector = TextDetector()

# df_csv['has_text'] = df_csv['image_path'].apply(detector.detect_text)
df_csv['has_text'] = df_csv['image_path'].progress_apply(detector.detect_text)

df_csv

In [ ]:
result_path = './csv_data/data_' + topic_name + '.csv'
df_csv.to_csv(result_path)

In [ ]:
# filter out those images without text
result_path = './csv_data/data_' + topic_name + '.csv'
df_csv = pd.read_csv(result_path, index_col=0)

df_csv_cleaned = df_csv[df_csv.has_text.eq(1)]
df_csv_cleaned.drop(columns=['has_text'], inplace=True)
df_csv_cleaned.reset_index(drop=True, inplace=True)

result_cleaned_path = './csv_data/data_' + topic_name + '_cleaned.csv'
df_csv_cleaned.to_csv(result_cleaned_path)

In [ ]:
df_csv_cleaned

### Count the tweets in different hashtags after cleaning

In [ ]:
result_cleaned_path = './csv_data/data_' + topic_name + '_cleaned.csv'
df_csv_cleaned = pd.read_csv(result_cleaned_path, index_col=0)

In [ ]:
# count the tweets in different hashtags after cleaning
df_csv_cleaned['hashtag'].value_counts()

### Filter out duplicate images

In [ ]:
result_cleaned_path = './csv_data/data_' + topic_name + '_cleaned.csv'
cleaned_data = pd.read_csv(result_cleaned_path, index_col=0)
imagePaths = cleaned_data['image_path'].to_list()

In [ ]:
from remove_duplicates import duplicate_detector
no_duplicate_paths = duplicate_detector(imagePaths)

In [ ]:
tqdm.pandas()
cleaned_data['no_dups'] = cleaned_data['image_path'].apply(lambda x:1 if x in no_duplicate_paths else 0)
df_csv_cleaned = cleaned_data[cleaned_data.no_dups.eq(1)]
df_csv_cleaned.drop(columns=['no_dups'], inplace=True)
df_csv_cleaned.reset_index(drop=True, inplace=True)

df_csv_cleaned

In [ ]:
result_cleaned_path = './csv_data/data_' + 'AsianHate' + '_cleaned_nodups.csv'
df_csv_cleaned.to_csv(result_cleaned_path)

In [ ]:
df_csv_cleaned['hashtag'].value_counts()